In [1]:

import torch
from dotenv import load_dotenv
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from transformers import AutoTokenizer
from transformers import GenerationConfig

load_dotenv()

True

In [2]:


original_model_name = 'google/gemma-3-270m-it'

new_model = AutoModelForCausalLM.from_pretrained(
    '/home/ivan/git/not-a-legitimate-stand-up/train/joke-generator',
    torch_dtype=torch.bfloat16,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=getattr(torch, 'float16'),
        bnb_4bit_use_double_quant=True,
    ),
    attn_implementation='flash_attention_2',
    device_map="cuda:0",
)

tokenizer = AutoTokenizer.from_pretrained(
    original_model_name,
    use_default_system_prompt=False,
    device_map=new_model.device,
)

generation_params = GenerationConfig(
    max_new_tokens=1024,
    temperature=1.2,
    top_k=25,
    top_p=1,
    repetition_penalty=1.1,
    eos_token_id=[1, 107],
    do_sample=True,
)

original_model = AutoModelForCausalLM.from_pretrained(
    original_model_name,
    trust_remote_code=True,
    attn_implementation='eager'
).to("cuda:0")

In [11]:
input_ids = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": [{"type": "text",
                         "text": "Output ONLY one short joke in Ukrainian language. Do NOT add other text."}, ]
        },
        {'role': "user", 'content': f'Розкажи жарт'}
    ],
    return_tensors='pt',
    add_generation_prompt=True,
    return_dict=True,
    return_full_text=False,
).to(new_model.device)
outputs_orig = original_model.generate(**input_ids, generation_config=generation_params)
outputs = new_model.generate(**input_ids, generation_config=generation_params)

joke_orig = tokenizer.decode(outputs_orig[0])
joke = tokenizer.decode(outputs[0])

print(joke_orig)
print('-------------')
print(joke)

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<bos><start_of_turn>user
Output ONLY one short joke in Ukrainian language. Do NOT add other text.

Розкажи жарт<end_of_turn>
<start_of_turn>model
Одяс у кулінарний майстермент, який збігує банку з великим обсягом товарів, а потім збирають на спеціальній фермі, яка робить з нього величезну ціну. На жаль, це все забагато коштує гроші!

-------------
<bos><start_of_turn>user
Output ONLY one short joke in Ukrainian language. Do NOT add other text.

Розкажи жарт<end_of_turn>
<start_of_turn>model
Жарт-жарт... Да як же, я так здею з ним не розбирався на бак, звичайно що не можехте! Говно слібається і мене в бак продимуєте на ножальну.</strong>

